In [61]:
# read diagnostics and parameter solutions of different calibrations
import numpy as np
import os
root_dir = '/glade/u/home/hongli/work/sharp/basins/scripts_hongli/'
os.chdir(root_dir)
output_folder='step2_calib_summary'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
outputfile = 'Calib_summary.txt'

calib_num = 6
param_num = 8
diag = []
title = 'Trial'

# read raw model results
raw_dir = os.path.join('../tayprk_huc12_metsim','Diagnostics.txt')
data = np.loadtxt(raw_dir, skiprows=1, usecols=0)
data_slice = ['no calib', float(data)]
data_slice.extend([1.0]*param_num)
diag.append(data_slice)

# read calibration results
for i in range(calib_num):
    calib_dir = os.path.join('../tayprk_calib_'+str(i+1)+'/', 'DDS.out')    
    data_slice = ['calib_trial_'+str(i+1)]    
    
    with open(calib_dir, 'r') as f:
         for iline, line in enumerate(f):
            line = line.strip()
            if line and 'Optimal Parameter Set' in line:
                iline_start = iline
                iline_end = iline_start + 1 + param_num 
                break
    with open(calib_dir, 'r') as f:
         for iline, line in enumerate(f):
            line = line.strip()
            if iline > iline_start and iline <= iline_end:
                splits = line.split(':')
                if iline == iline_start+1:
                    data_slice.append((-1)*float(splits[1]))
                else:
                    data_slice.append(float(splits[1]))
                if i == 0:
                    if iline == iline_start+1:
                        title=title+'\t'+'KGE'
                    else:
                        title=title+'\t'+splits[0]
            elif iline > iline_end:
                break
                    
    diag.append(data_slice)            

# write output
f = open(os.path.join(root_dir,output_folder,outputfile), 'w+')
f.write(title+'\n')
for diag_row in diag:
    f.write('%s\t' %diag_row[0])
    for i in range(len(diag_row)-1):
        f.write('%.2f\t' %diag_row[i+1])
    f.write('\n')
f.close()
